In [ ]:
#Pandas is a datascience library 
# Requests is for the API of the data 
# JSON was used for the taking 
# Folium is a map making library
import pandas as pd
from pandas import DataFrame
import requests
import json
!pip install folium 
!pip install folium --upgrade
import matplotlib.pyplot as plt

     |████████████████████████████████| 102kB 5.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
#This is to check for permission for the API 
response = requests.get("https://opendata.arcgis.com/datasets/f1cd175a268e46c9b8517dfe8e2fa931_29.geojson")
response.status_code

200

In [ ]:
#This translates the data into a json object. 
data2 = response.json()
print(str(data2)[:1000])

{'type': 'FeatureCollection', 'name': 'Transit_2020', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'TransitID': 2453, 'Stop': '10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location', 'Route': '07 A', 'Date_Time': '2020/01/18 12:09:00+00', 'Count': 1, 'Fare': 0, 'FareCategory': 'UVA Academic', 'PaymentType': 'Barcode-3rd Party Media', 'Latitude': 38.0363, 'Longitude': -78.5078}, 'geometry': None}, {'type': 'Feature', 'properties': {'TransitID': 2454, 'Stop': '10472 - Emmet St at Central Grounds Please refer to the Latitude/Longitude for location', 'Route': '07 A', 'Date_Time': '2020/01/18 12:09:00+00', 'Count': 1, 'Fare': 0, 'FareCategory': 'UVA Academic', 'PaymentType': 'Barcode-3rd Party Media', 'Latitude': 38.0363, 'Longitude': -78.5078}, 'geometry': None}, {'type': 'Feature', 'properties': {'TransitID': 2455, 'Stop': '10472 - Emmet St at Central Grounds Please refer

In [ ]:
#Normalizes the data 
data2 = pd.io.json.json_normalize(data2["features"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [ ]:
#This renames the columns of the data frame 
data2.rename(columns={
    'properties.Count': 'Count',
    'properties.Date_Time': 'Date/Time',
    'properties.Fare': 'Fare',
    'properties.FareCategory': 'Fare Category',
    'properties.Latitude': 'Latitude',
    'properties.Longitude': 'Longitude',
    'properties.PaymentType': 'PaymentType',
    'properties.Route': 'route',
    'properties.Stop': 'Stop',
    'properties.TransitID': 'TransitID'
}, inplace=True)

In [ ]:
data2

,type,geometry,TransitID,Stop,route,Date/Time,Count,Fare,Fare Category,PaymentType,Latitude,Longitude
0,Feature,None,2453,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 12:09:00+00,1,0,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5078
1,Feature,None,2454,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 12:09:00+00,1,0,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5078
2,Feature,None,2455,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 12:09:00+00,1,0,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5078
3,Feature,None,2456,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 13:52:00+00,1,0,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5079
4,Feature,None,2457,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 13:52:00+00,1,0,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5079
...,...,...,...,...,...,...,...,...,...,...,...,...
215402,Feature,None,217855,16596 - Fashion Square Mall,11,2020/02/18 14:03:00+00,1,0,30 Day Full,Smart Media-Time Based,38.0775,-78.4736
215403,Feature,None,217856,16596 - Fashion Square Mall,11,2020/02/18 14:58:00+00,1,0,Youth Smart,Smart Media-Date Based,38.0775,-78.4736
215404,Feature,None,217857,16596 - Fashion Square Mall,11,2020/02/18 15:57:00+00,1,0,30 Day Full,Smart Media-Time Based,38.0775,-78.4736
215405,Feature,None,217858,16596 - Fashion Square Mall,11,2020/02/18 16:57:00+00,1,0,30 Day Full,Smart Media-Time Based,38.0774,-78.4737


In [ ]:
import folium
from folium import plugins

In [ ]:
#215407
from folium import plugins
from folium.plugins import MarkerCluster


busmap = folium.Map(location=[38.0363, -78.5078], zoom_start=12)

original_copy = data2 

something = original_copy[["Stop","Latitude","Longitude"]]
#Only unique stops are counted 
something = something.drop_duplicates(subset=["Stop"])
something


#Inputting all data points 
for index,row in something.iterrows():
   lat = row["Latitude"]
   lon = row["Longitude"]
   name = row["Stop"]
   folium.Marker([lat,lon],popup=name).add_to(busmap)


In [ ]:
something

,Stop,Latitude,Longitude
0,10472 - Emmet St at Central Grounds Please ref...,38.0363,-78.5078
176,11015 - East Water St at 3rd St SE,38.0296,-78.4802
2099,11036 - West Water St at Omni,38.0310,-78.4844
5396,11043 - Omni Hotel at Ridge McIntire Rd,38.0317,-78.4849
5717,11058 - West Main Street at 4th St,38.0305,-78.4855
...,...,...,...
151089,15049 - Grove Road at Meadowbrook Heights (eas...,38.0558,-78.4829
156214,15246 - 5th Street Ext at Moores Creek (northb...,38.0099,-78.5063
180037,19459 - 10th St Nw at Page St northbound,38.0354,-78.4933
180051,19526 - Grove Rd at Concord Dr Eastbound,38.0541,-78.4791


In [ ]:
busmap

In [ ]:
thinking = original_copy.groupby("Stop").size()
#Checks how often theses stops were visited 
original_copy['freq']=original_copy.groupby(by='Stop')['Stop'].transform('count')
pop = original_copy.drop_duplicates(subset=["Stop"])
pop

,type,geometry,TransitID,Stop,route,Date/Time,Count,Fare,Fare Category,PaymentType,Latitude,Longitude,freq
0,Feature,None,2453,10472 - Emmet St at Central Grounds Please ref...,07 A,2020/01/18 12:09:00+00,1,0,UVA Academic,Barcode-3rd Party Media,38.0363,-78.5078,1490
176,Feature,None,2629,11015 - East Water St at 3rd St SE,12,2020/01/12 15:52:00+00,1,0,ADA,Manual,38.0296,-78.4802,5788
2099,Feature,None,4552,11036 - West Water St at Omni,01,2020/01/02 09:33:00+00,1,0,ADA,Manual,38.0310,-78.4844,5247
5396,Feature,None,7849,11043 - Omni Hotel at Ridge McIntire Rd,01,2020/01/02 18:01:00+00,1,0,30 Day Full,Smart Media-Time Based,38.0317,-78.4849,605
5717,Feature,None,8170,11058 - West Main Street at 4th St,07 A,2020/01/04 15:05:00+00,1,0,24 Hour Full,Barcode-Time Based,38.0305,-78.4855,335
...,...,...,...,...,...,...,...,...,...,...,...,...,...
151089,Feature,None,153542,15049 - Grove Road at Meadowbrook Heights (eas...,09,2020/02/24 13:05:00+00,1,1,One Ride Full,Cash-Value,38.0558,-78.4829,1
156214,Feature,None,158667,15246 - 5th Street Ext at Moores Creek (northb...,03 A,2020/02/01 20:52:00+00,1,0,30 Day Full,Smart Media-Time Based,38.0099,-78.5063,3
180037,Feature,None,182490,19459 - 10th St Nw at Page St northbound,09,2020/02/03 13:45:00+00,1,1,One Ride Full,Cash-Value,38.0354,-78.4933,4
180051,Feature,None,182504,19526 - Grove Rd at Concord Dr Eastbound,09,2020/02/03 14:00:00+00,1,0,24 Hour Full,Barcode-Time Based,38.0541,-78.4791,6


# Popularity

In [ ]:
testmap = folium.Map(location=[38.0363, -78.5078], zoom_start=12)

for index,row in pop.iterrows():
   lat = row["Latitude"]
   lon = row["Longitude"]
   name = row["Stop"]
   freq = row["freq"]
   folium.Marker([lat,lon],popup=f"Address:{name} Popularity:{freq}").add_to(testmap)


In [ ]:
testmap

In [ ]:
testmap.save("map_withpop.html")